In [6]:
# import libraries
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

from scipy.integrate import odeint, solve_ivp
from scipy.fft import fft


import shap as shap
try:
    import lime
    import lime.lime_tabular    
except ImportError:
    pass

# Machine Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import Sequence
from tensorflow import keras

# for reproducibility of this notebook:
rng = np.random.RandomState(42)
tf.random.set_seed(42)
np.random.seed(42)


In [7]:
class Duffing():
    """
        Class for the Duffing Oscillator
    """
    def __init__(self, parameters = {'alpha': [0.3], 'beta': [-0.1], 'gamma': [0.37], 'delta': [0.3], 'omega': [1.2]}, 
                 labels = ['xt','vt'], features = ['x0','v0', 't', 'rand', 'energy'], scaler = None):
        """
            Define Parameter Configuration to Model

            Parameters
            ----------
            alpha : float, linear stiffness
            beta  : float, non linearity in the restoring force
            gamma : float, amplitude of the periodic driving force
            delta : float, amount of damping
            omega : float, angular frequency of the periodic driving force
        """   

        self.labels = labels
        self.features = features + [param for param in parameters]
        self.scaler = scaler
        self.suffix = "params_"+str(parameters['alpha'])+"_"+str(parameters['beta'])+"_"+str(parameters['gamma'])+"_"+str(parameters['delta'])+"_"+str(parameters['omega'])
        self.parameter_grid = ParameterGrid(parameters)
        self.parameters = list(self.parameter_grid)[0]
        

            
        
    def eom(self, t, u):
        """
            Duffing Oscillator Equation of Motion

            ddx + delta * dx + alpha * x + beta * x**3 = gamma * cos(omega * t)

            Input
            ----------
            u : vector of length 2, (x,v)
                Position and Velocity at time t
            t : float, the time t

            Returns
            ----------
            [dx,ddx] : Tuple, Time derivatives of 
                        position and velocity at time t
        """
        x, dx = u[0], u[1]
        ddx = (self.parameters['gamma'] * np.cos(self.parameters['omega'] * t) - (self.parameters['delta'] * dx + self.parameters['alpha'] * x + self.parameters['beta'] * x**3))

        return [dx,ddx]
    
    def energy(self, x, v):
        return 0.5*v**2 + 0.5*self.parameters['alpha']*x**2 +0.25*self.parameters['beta']*x**4

    def termination_event(self, t, y):
        """
            Stops Numerical Integration once points wander too far away
        """
        return (np.abs(y[0]) - 10)*(np.abs(y[1]) - 10)
    termination_event.terminal = True


    def generate(self, num_samples = int(5e1), samples=10, end_time=100, gridded=False, num_gammas = 1):
        """
            Generates training samples using scipy.integrate.odeint
            to calculate the temporal evolution of a Duffing system.
    
            Samples randomly from x0 in [-2,2], v0 in [-1,1].
    
            For each set of initial conditions we generate a trajectory.
            The trajectory is randomly sampled to generate training
            pairs: X = (x0,v0,t), y = (xt,vt)
    
            Input
            ----------
            num_samples : int, number of training
                            samples to be generated
    
            Returns
            ----------
            X : array((num_samples,3)), each entry in the array
                is a training sample (x0,v0,t)
            y : array((num_samples,2)), each entry in the array
                is a target sample (xt,vt)
        """
        if num_gammas > 1:
            self.num_gammas = num_gammas
            self.suffix = self.suffix + '_gamma_on'
            self.features = self.features + ['gamma']
        self.scaler = None
        #Define bounds of the sampling
        x_min = -2
        x_max = 2
        v_min = -2
        v_max = 2
        #Initialise the output arrays        
        X = np.empty((num_samples*samples*len(list(self.parameter_grid)), len(np.hstack((self.features, self.labels)))))
        #Define the t_range to draw from
        t_range = np.linspace(0, end_time, 100, endpoint=False)
        t_vals = np.sort(np.random.choice(t_range, size = samples, replace=False))

        #Generate num_samples samples
        for i in tqdm(range(num_samples), desc="Generating Data…", ascii=False, ncols=75):
            #Generate random starting positions
            x0 = (x_max - x_min) * np.random.random_sample() + x_min
            v0 = (v_max - v_min) * np.random.random_sample() + v_min
            for j, dict_param in enumerate(self.parameter_grid):
                # Do something with the current parameter combination in ``dict_``
                self.parameters = dict_param
                #Generate a trajectory
                trajectory = solve_ivp(self.eom, [0, end_time], [x0,v0], t_eval = t_vals, events = [self.termination_event])
                traj_cutoff =  samples - len(trajectory.y[0])
                traj_x = np.append(trajectory.y[0].reshape(-1,1), 10.0*np.ones(traj_cutoff).reshape(-1,1))
                traj_v = np.append(trajectory.y[1].reshape(-1,1), 10.0*np.ones(traj_cutoff).reshape(-1,1))
                val_range_low = i*samples*len(list(self.parameter_grid))+j*samples
                val_range_high = i*samples*len(list(self.parameter_grid))+(j+1)*samples
                X[val_range_low:val_range_high,:] = np.hstack((x0*np.ones(samples).reshape(-1,1), 
                                               v0*np.ones(samples).reshape(-1,1),
                                               t_vals.reshape(-1,1),
                                               np.random.uniform(-1,1,samples).reshape(-1,1),
                                               self.energy(x0,v0)*np.ones(samples).reshape(-1,1),
                                               np.array([[dict_param[i] for i in dict_param] for i in range(samples)]).reshape((-1,len(dict_param))),
                                               traj_x.reshape(-1,1), 
                                               traj_v.reshape(-1,1)))
        
        self.X_df = pd.DataFrame(X, columns = np.hstack((self.features, self.labels)))
        return self.X_df

    def scale_features(self):
        if self.scaler == None:
            self.scaler = MinMaxScaler(feature_range=[0,1])
            self.X_df[self.features] = self.scaler.fit_transform(self.X_df[self.features].values)
        else: return



    def predict(self, X):
        if self.scaler == None:
            self.scale_features()
        if type(X) == pd.core.frame.DataFrame:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X.values), columns=X.columns)
        elif type(X) == np.ndarray:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X), columns=self.features)

        y = np.ones((np.shape(X_temp)[0], 2))
        for i in range(0,np.shape(X_temp)[0]):
            for j in self.parameters:
                self.parameters[j] = X_temp[j].iloc[i]
            traj = solve_ivp(self.eom, [0, X_temp['t'].iloc[i]], [X_temp['x0'].iloc[i], X_temp['v0'].iloc[i]], 
                            t_eval = None, events = [self.termination_event])
            y[i] = [traj.y[0][-1], traj.y[1][-1]]
            
        return y

    def predict_x(self, X):
        if self.scaler == None:
            self.scale_features()
        if type(X) == pd.core.frame.DataFrame:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X.values), columns=X.columns)
        elif type(X) == np.ndarray:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X), columns=self.features)

        y = np.ones((np.shape(X_temp)[0], 1))
        for i in range(0,np.shape(X_temp)[0]):
            for j in self.parameters:
                self.parameters[j] = X_temp[j].iloc[i]
            traj = solve_ivp(self.eom, [0, X_temp['t'].iloc[i]], [X_temp['x0'].iloc[i], X_temp['v0'].iloc[i]], 
                            t_eval = None, events = [self.termination_event])
            y[i] = traj.y[0][-1]

        return y

    def predict_v(self, X):
        if self.scaler == None:
            self.scale_features()
        if type(X) == pd.core.frame.DataFrame:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X.values), columns=X.columns)
        elif type(X) == np.ndarray:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X), columns=self.features)

        y = np.ones((np.shape(X_temp)[0], 1))
        for i in range(0,np.shape(X_temp)[0]):
            for j in self.parameters:
                self.parameters[j] = X_temp[j].iloc[i]
            traj = solve_ivp(self.eom, [0, X_temp['t'].iloc[i]], [X_temp['x0'].iloc[i], X_temp['v0'].iloc[i]], 
                            t_eval = None, events = [self.termination_event])
            y[i] = traj.y[1][-1]

            
        return y

        
    def vals_to_df(self, values, data, save=False, explainer = "lime", suffix = None):
        xt_atts = values[0]
        vt_atts = values[1]
        data = data
        
        param_array = np.array(np.ones((data.shape[0], 5)))
        for i, param in enumerate(self.parameters):
            param_array[:,i] = self.parameters[param]*np.ones((data.shape[0]))

        df = pd.DataFrame(xt_atts, columns = [self.labels[0] + "_" + i for i in self.features])
        df = df.join(pd.DataFrame(vt_atts, columns = [self.labels[1] + "_" + i for i in self.features]))
        df = df.join(pd.DataFrame(data.values, columns = self.features))
        df = df.join(pd.DataFrame(param_array, columns = self.parameters.keys()))
        df.insert(df.shape[1], 'explainer' ,[explainer for _ in range(df.shape[0])])
        if save:
            df.to_csv("Results/"+explainer+"/"+explainer+"_vals_"+suffix+".csv")
        return df



In [13]:
        
"""
Define Parameter Configuration to Model

    Parameters
    ----------
    alpha : float, linear stiffness
    beta  : float, non linearity in the restoring force
    gamma : float, amplitude of the periodic driving force
    delta : float, amount of damping
    omega : float, angular frequency of the periodic driving force
"""   
alpha = [-1.0]
beta = [1.0]
gamma = [0.2, 0.28, 0.37, 0.5, 0.65]
delta = [0.3, 1.0]
omega = [0.3, 1.2]

def energy(x, v):
    return 0.5*v**2 + 0.5*alpha*x**2 +0.25*beta*x**4

parameters_now = {'alpha': alpha, 'beta': beta, 'gamma': gamma, 'delta': delta, 'omega':omega}
duffing = Duffing(parameters = parameters_now)
eom = duffing.eom
suffix = duffing.suffix









In [14]:
"""
Define and Create Model
"""

def MLModel():
    opt = Adam(learning_rate=0.001, beta_1=0.7)
    loss='mse'
    model = Sequential([
        layers.Flatten(),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='sigmoid'),
        layers.Dense(32, activation='tanh'),
        layers.Dense(2)            
    ])
    model.compile(optimizer=opt, loss=loss)
    return model

# Create a basic model instance
model = MLModel()

In [15]:
end_time = 100
duffing.generate(10, samples = 100, end_time = end_time)
duffing.scale_features()
X = duffing.X_df[duffing.features]
y = duffing.X_df[duffing.labels]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

Generating Data…: 100%|████████████████████| 10/10 [00:05<00:00,  1.82it/s]


In [16]:
X

,x0,v0,t,rand,energy,alpha,beta,gamma,delta,omega
0,0.033283,0.897161,0.000000,0.330888,0.568410,0.0,0.0,0.0,0.0,0.0
1,0.033283,0.897161,0.010101,0.264146,0.568410,0.0,0.0,0.0,0.0,0.0
2,0.033283,0.897161,0.020202,0.750172,0.568410,0.0,0.0,0.0,0.0,0.0
3,0.033283,0.897161,0.030303,0.261992,0.568410,0.0,0.0,0.0,0.0,0.0
4,0.033283,0.897161,0.040404,0.105483,0.568410,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
19995,0.539354,0.496351,0.959596,0.613517,0.119022,0.0,0.0,1.0,1.0,1.0
19996,0.539354,0.496351,0.969697,0.422689,0.119022,0.0,0.0,1.0,1.0,1.0
19997,0.539354,0.496351,0.979798,0.861403,0.119022,0.0,0.0,1.0,1.0,1.0
19998,0.539354,0.496351,0.989899,0.237179,0.119022,0.0,0.0,1.0,1.0,1.0


In [20]:
"""
Train Model
"""
#model.build()
# Display the model's architecture
#model.summary()



# Create a callback that saves the model's weights
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25),
             tf.keras.callbacks.EarlyStopping(monitor='loss', patience=25)]


# pipe = make_pipeline(scaler, model)

history=model.fit(X_train, y_train, steps_per_epoch=None, epochs=500, 
                  validation_split=0.2, batch_size=1024, shuffle=True, callbacks=None, verbose=1)

loss = model.evaluate(X_test, y_test, verbose=1)
print("Trained model, loss: {:5.2f}%".format(loss))

model.save("Models/param_grid_model")

Epoch 1/500
15/15 [==============================] - 0s 13ms/step - loss: 0.1926 - val_loss: 0.2150
Epoch 2/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1919 - val_loss: 0.1967
Epoch 3/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1904 - val_loss: 0.1943
Epoch 4/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1855 - val_loss: 0.2063
Epoch 5/500
15/15 [==============================] - 0s 12ms/step - loss: 0.1863 - val_loss: 0.1892
Epoch 6/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1813 - val_loss: 0.2303
Epoch 7/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1959 - val_loss: 0.1914
Epoch 8/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1850 - val_loss: 0.1911
Epoch 9/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1874 - val_loss: 0.1929
Epoch 10/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1846 - val_loss: 0.2330

15/15 [==============================] - 0s 11ms/step - loss: 0.1758 - val_loss: 0.2136
Epoch 83/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1767 - val_loss: 0.2000
Epoch 84/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1730 - val_loss: 0.2474
Epoch 85/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1899 - val_loss: 0.2143
Epoch 86/500
15/15 [==============================] - 0s 12ms/step - loss: 0.1814 - val_loss: 0.2010
Epoch 87/500
15/15 [==============================] - 0s 12ms/step - loss: 0.1764 - val_loss: 0.1895
Epoch 88/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1709 - val_loss: 0.2276
Epoch 89/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1859 - val_loss: 0.1992
Epoch 90/500
15/15 [==============================] - 0s 12ms/step - loss: 0.1731 - val_loss: 0.1890
Epoch 91/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1681 - val_loss: 0.1938
Epo

15/15 [==============================] - 0s 12ms/step - loss: 0.1496 - val_loss: 0.1827
Epoch 163/500
15/15 [==============================] - 0s 16ms/step - loss: 0.1473 - val_loss: 0.1709
Epoch 164/500
15/15 [==============================] - 0s 12ms/step - loss: 0.1393 - val_loss: 0.1881
Epoch 165/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1534 - val_loss: 0.1788
Epoch 166/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1437 - val_loss: 0.1765
Epoch 167/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1346 - val_loss: 0.1992
Epoch 168/500
15/15 [==============================] - 0s 13ms/step - loss: 0.1467 - val_loss: 0.1797
Epoch 169/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1342 - val_loss: 0.1751
Epoch 170/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1365 - val_loss: 0.1780
Epoch 171/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1385 - val_loss: 0

15/15 [==============================] - 0s 11ms/step - loss: 0.1682 - val_loss: 0.1895
Epoch 243/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1368 - val_loss: 0.1814
Epoch 244/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1350 - val_loss: 0.1820
Epoch 245/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1395 - val_loss: 0.1778
Epoch 246/500
15/15 [==============================] - 0s 12ms/step - loss: 0.1244 - val_loss: 0.1961
Epoch 247/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1369 - val_loss: 0.1795
Epoch 248/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1379 - val_loss: 0.1868
Epoch 249/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1298 - val_loss: 0.1633
Epoch 250/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1160 - val_loss: 0.1699
Epoch 251/500
15/15 [==============================] - 0s 11ms/step - loss: 0.1212 - val_loss: 0

15/15 [==============================] - 0s 11ms/step - loss: 0.0893 - val_loss: 0.1563
Epoch 323/500
15/15 [==============================] - 0s 12ms/step - loss: 0.0896 - val_loss: 0.1605
Epoch 324/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0947 - val_loss: 0.1566
Epoch 325/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0913 - val_loss: 0.1553
Epoch 326/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0888 - val_loss: 0.1496
Epoch 327/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0861 - val_loss: 0.1570
Epoch 328/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0899 - val_loss: 0.1495
Epoch 329/500
15/15 [==============================] - 0s 13ms/step - loss: 0.0833 - val_loss: 0.1637
Epoch 330/500
15/15 [==============================] - 0s 12ms/step - loss: 0.0858 - val_loss: 0.1472
Epoch 331/500
15/15 [==============================] - 0s 12ms/step - loss: 0.0809 - val_loss: 0

15/15 [==============================] - 0s 11ms/step - loss: 0.0867 - val_loss: 0.1544
Epoch 403/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0817 - val_loss: 0.1472
Epoch 404/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0770 - val_loss: 0.1621
Epoch 405/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0817 - val_loss: 0.1569
Epoch 406/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0795 - val_loss: 0.1571
Epoch 407/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0844 - val_loss: 0.1582
Epoch 408/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0817 - val_loss: 0.1610
Epoch 409/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0859 - val_loss: 0.1602
Epoch 410/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0821 - val_loss: 0.1608
Epoch 411/500
15/15 [==============================] - 0s 12ms/step - loss: 0.0788 - val_loss: 0

15/15 [==============================] - 0s 12ms/step - loss: 0.0773 - val_loss: 0.1707
Epoch 483/500
15/15 [==============================] - 0s 12ms/step - loss: 0.0750 - val_loss: 0.1546
Epoch 484/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0666 - val_loss: 0.1656
Epoch 485/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0736 - val_loss: 0.1586
Epoch 486/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0708 - val_loss: 0.1598
Epoch 487/500
15/15 [==============================] - 0s 11ms/step - loss: 0.0692 - val_loss: 0.1618
Epoch 488/500
15/15 [==============================] - 0s 12ms/step - loss: 0.0673 - val_loss: 0.1628
Epoch 489/500
15/15 [==============================] - 0s 12ms/step - loss: 0.0627 - val_loss: 0.1767
Epoch 490/500
15/15 [==============================] - 0s 12ms/step - loss: 0.0694 - val_loss: 0.1817
Epoch 491/500
15/15 [==============================] - 0s 12ms/step - loss: 0.0839 - val_loss: 0

In [ ]:
def ml_x(X):
    return model.predict(X)[0]
def ml_v(X):
    return model.predict(X)[1]

explainers = ["kernel", "sampling", "lime", "numeric"]
true_lime = [duffing.predict_x, duffing.predict_v]
ml_lime = [ml_x, ml_v]
models = {"true" : duffing.predict, "ml" : model.predict}
lime_models = {"true" : true_lime, "ml" : ml_lime}


background = shap.sample(X_test, 100)
choice = X.iloc[np.sort(np.random.choice(X_test.shape[0], 100, replace =False))]


big_df = pd.DataFrame()
for model in models:
    for explainer in explainers:
        print(explainer + model)
        if explainer == "kernel":
            temp_explainer = shap.KernelExplainer(models[model], background)
            temp_vals = temp_explainer.shap_values(choice)
        elif explainer == "sampling":
            temp_explainer = shap.SamplingExplainer(models[model], background)
            temp_vals = temp_explainer.shap_values(choice)
        elif explainer == "lime":
            temp_explainer = MyLime(lime_models[model], choice, mode='regression')
            temp_vals = temp_explainer.attributions(choice)
        elif explainer == "numeric":
            temp_explainer = NumericExplainer(models[model], duffing.features, duffing.labels, h = 0.001)
            temp_vals = temp_explainer.feature_att(choice)
        else:
            print("not a valid explainer type")
        big_df = big_df.append(duffing.vals_to_df(temp_vals, 
                                                        choice, save=False, explainer = explainer, suffix = suffix+model))
        
        
big_df.to_csv("Results/explainer_dataframe_full.csv")  


In [84]:
# import libraries
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

from scipy.integrate import odeint, solve_ivp
from scipy.fft import fft


import shap as shap
try:
    import lime
    import lime.lime_tabular    
except ImportError:
    pass

# Machine Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import Sequence
from tensorflow import keras

# for reproducibility of this notebook:
rng = np.random.RandomState(42)
tf.random.set_seed(42)
np.random.seed(42)


class Duffing():
    """
        Class for the Duffing Oscillator
    """
    def __init__(self, parameters = {'alpha': [0.3], 'beta': [-0.1], 'gamma': [0.37], 'delta': [0.3], 'omega': [1.2]}, 
                 labels = ['xt','vt'], features = ['x0','v0', 't', 'rand', 'energy'], scaler = None, passed_grid = False):
        """
            Define Parameter Configuration to Model

            Parameters
            ----------
            alpha : float, linear stiffness
            beta  : float, non linearity in the restoring force
            gamma : float, amplitude of the periodic driving force
            delta : float, amount of damping
            omega : float, angular frequency of the periodic driving force
        """   
        self.labels = labels

        self.features = features + [param for param in parameters]
        self.all_features = self.features + [param for param in parameters]
        self.scaler = scaler
        self.suffix = "params_"+str(parameters['alpha'])+"_"+str(parameters['beta'])+"_"+str(parameters['gamma'])+"_"+str(parameters['delta'])+"_"+str(parameters['omega'])
        self.passed_grid = passed_grid
        if passed_grid:
            self.parameter_grid = ParameterGrid(parameters)
            self.parameters = list(self.parameter_grid)[0]
            self.param_len = len(self.parameter_grid)
        else:
            self.parameters = parameters
            self.param_len = 1
        

            
        
    def eom(self, t, u):
        """
            Duffing Oscillator Equation of Motion

            ddx + delta * dx + alpha * x + beta * x**3 = gamma * cos(omega * t)

            Input
            ----------
            u : vector of length 2, (x,v)
                Position and Velocity at time t
            t : float, the time t

            Returns
            ----------
            [dx,ddx] : Tuple, Time derivatives of 
                        position and velocity at time t
        """
        x, dx = u[0], u[1]
        ddx = (self.parameters['gamma'] * np.cos(self.parameters['omega'] * t) - (self.parameters['delta'] * dx + self.parameters['alpha'] * x + self.parameters['beta'] * x**3))

        return [dx,ddx]
    
    def energy(self, x, v):
        return 0.5*v**2 + 0.5*self.parameters['alpha']*x**2 +0.25*self.parameters['beta']*x**4

    def termination_event(self, t, y):
        """
            Stops Numerical Integration once points wander too far away
        """
        return (np.abs(y[0]) - 10)*(np.abs(y[1]) - 10)
    termination_event.terminal = True


    def generate(self, num_samples = int(5e1), samples=10, end_time=100, gridded=False, num_gammas = 1):
        """
            Generates training samples using scipy.integrate.odeint
            to calculate the temporal evolution of a Duffing system.
    
            Samples randomly from x0 in [-2,2], v0 in [-1,1].
    
            For each set of initial conditions we generate a trajectory.
            The trajectory is randomly sampled to generate training
            pairs: X = (x0,v0,t), y = (xt,vt)
    
            Input
            ----------
            num_samples : int, number of training
                            samples to be generated
    
            Returns
            ----------
            X : array((num_samples,3)), each entry in the array
                is a training sample (x0,v0,t)
            y : array((num_samples,2)), each entry in the array
                is a target sample (xt,vt)
        """
        self.scaler = None
        #Define bounds of the sampling
        x_min = -2
        x_max = 2
        v_min = -2
        v_max = 2
        #Initialise the output arrays        
        X = np.empty((num_samples*samples*self.param_len, len(np.hstack((self.features, self.labels)))))
        #Define the t_range to draw from
        t_range = np.linspace(0, end_time, 100, endpoint=False)
        t_vals = np.sort(np.random.choice(t_range, size = samples, replace=False))

        #Generate num_samples samples
        for i in tqdm(range(num_samples), desc="Generating Data…", ascii=False, ncols=75):
            #Generate random starting positions
            x0 = (x_max - x_min) * np.random.random_sample() + x_min
            v0 = (v_max - v_min) * np.random.random_sample() + v_min
            if self.passed_grid:
                for j, dict_param in enumerate(self.parameter_grid):
                    # Do something with the current parameter combination in ``dict_``
                    self.parameters = dict_param
                    #Generate a trajectory
                    trajectory = solve_ivp(self.eom, [0, end_time], [x0,v0], t_eval = t_vals, events = [self.termination_event])
                    traj_cutoff =  samples - len(trajectory.y[0])
                    traj_x = np.append(trajectory.y[0].reshape(-1,1), 10.0*np.ones(traj_cutoff).reshape(-1,1))
                    traj_v = np.append(trajectory.y[1].reshape(-1,1), 10.0*np.ones(traj_cutoff).reshape(-1,1))
                    val_range_low = i*samples*self.param_len+j*samples
                    val_range_high = i*samples*self.param_len+(j+1)*samples
                    X[val_range_low:val_range_high,:] = np.hstack((x0*np.ones(samples).reshape(-1,1), 
                                                   v0*np.ones(samples).reshape(-1,1),
                                                   t_vals.reshape(-1,1),
                                                   np.random.uniform(-1,1,samples).reshape(-1,1),
                                                   self.energy(x0,v0)*np.ones(samples).reshape(-1,1),
                                                   np.array([[dict_param[i] for i in dict_param] for i in range(samples)]).reshape((-1,len(dict_param))),
                                                   traj_x.reshape(-1,1), 
                                                   traj_v.reshape(-1,1)))
            else:
                dict_param = self.parameters
                # Do something with the current parameter combination in ``dict_``
                self.parameters = dict_param
                #Generate a trajectory
                trajectory = solve_ivp(self.eom, [0, end_time], [x0,v0], t_eval = t_vals, events = [self.termination_event])
                traj_cutoff =  samples - len(trajectory.y[0])
                traj_x = np.append(trajectory.y[0].reshape(-1,1), 10.0*np.ones(traj_cutoff).reshape(-1,1))
                traj_v = np.append(trajectory.y[1].reshape(-1,1), 10.0*np.ones(traj_cutoff).reshape(-1,1))
                val_range_low = i*samples*self.param_len
                val_range_high = (i+1)*samples*self.param_len
                X[val_range_low:val_range_high,:] = np.hstack((x0*np.ones(samples).reshape(-1,1), 
                                               v0*np.ones(samples).reshape(-1,1),
                                               t_vals.reshape(-1,1),
                                               np.random.uniform(-1,1,samples).reshape(-1,1),
                                               self.energy(x0,v0)*np.ones(samples).reshape(-1,1),
                                               np.array([[dict_param[i] for i in dict_param] for i in range(samples)]).reshape((-1,len(dict_param))),
                                               traj_x.reshape(-1,1), 
                                               traj_v.reshape(-1,1)))
        
        self.X_df = pd.DataFrame(X, columns = np.hstack((self.features, self.labels)))
        return self.X_df

    def scale_features(self):
        if self.scaler == None:
            self.scaler = MinMaxScaler(feature_range=[0,1])
            self.X_df[self.features] = self.scaler.fit_transform(self.X_df[self.features].values)
        else: return



    def predict(self, X):
        if self.scaler == None:
            self.scale_features()
        if type(X) == pd.core.frame.DataFrame:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X.values), columns=X.columns)
        elif type(X) == np.ndarray:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X), columns=self.features)

        y = np.ones((np.shape(X_temp)[0], 2))
        for i in range(0,np.shape(X_temp)[0]):
            for j in self.parameters:
                self.parameters[j] = X_temp[j].iloc[i]
            traj = solve_ivp(self.eom, [0, X_temp['t'].iloc[i]], [X_temp['x0'].iloc[i], X_temp['v0'].iloc[i]], 
                            t_eval = None, events = [self.termination_event])
            y[i] = [traj.y[0][-1], traj.y[1][-1]]
            
        return y

    def predict_x(self, X):
        if self.scaler == None:
            self.scale_features()
        if type(X) == pd.core.frame.DataFrame:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X.values), columns=X.columns)
        elif type(X) == np.ndarray:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X), columns=self.features)

        y = np.ones((np.shape(X_temp)[0], 1))
        for i in range(0,np.shape(X_temp)[0]):
            for j in self.parameters:
                self.parameters[j] = X_temp[j].iloc[i]
            traj = solve_ivp(self.eom, [0, X_temp['t'].iloc[i]], [X_temp['x0'].iloc[i], X_temp['v0'].iloc[i]], 
                            t_eval = None, events = [self.termination_event])
            y[i] = traj.y[0][-1]

        return y

    def predict_v(self, X):
        if self.scaler == None:
            self.scale_features()
        if type(X) == pd.core.frame.DataFrame:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X.values), columns=X.columns)
        elif type(X) == np.ndarray:
            X_temp = pd.DataFrame(self.scaler.inverse_transform(X), columns=self.features)

        y = np.ones((np.shape(X_temp)[0], 1))
        for i in range(0,np.shape(X_temp)[0]):
            for j in self.parameters:
                self.parameters[j] = X_temp[j].iloc[i]
            traj = solve_ivp(self.eom, [0, X_temp['t'].iloc[i]], [X_temp['x0'].iloc[i], X_temp['v0'].iloc[i]], 
                            t_eval = None, events = [self.termination_event])
            y[i] = traj.y[1][-1]

            
        return y

        
    def vals_to_df(self, values, data, save=False, explainer = "lime", suffix = None):
        xt_atts = values[0]
        vt_atts = values[1]
        data = data
        
        param_array = np.array(np.ones((data.shape[0], 5)))
        for i, param in enumerate(self.parameters):
            param_array[:,i] = self.parameters[param]*np.ones((data.shape[0]))

        df = pd.DataFrame(xt_atts, columns = [self.labels[0] + "_" + i for i in self.features])
        df = df.join(pd.DataFrame(vt_atts, columns = [self.labels[1] + "_" + i for i in self.features]))
        df = df.join(pd.DataFrame(data.values, columns = self.features))
        df.insert(df.shape[1], 'explainer' ,[explainer for _ in range(df.shape[0])])
        if save:
            df.to_csv("Results/"+explainer+"/"+explainer+"_vals_"+suffix+".csv")
        return df

class NumericExplainer():
    """
        Pretty Brute force numerical gradient calculation for
        explainability of a known function
    """
    def __init__(self, f, features, labels,  h=0.01):
        """
            Initialises with some configurations for the gradient calculation
            as well as the function being differentiated.
            
            Inputs
            --------
            f : function that takes a pandas.DataFrame and outputs a 2d np.array.
            features : list of features in the pd.DataFrame for which we are to 
                differentiate f.
            labels : list of features in the np.array.
        """
        self.f = f
        self.features = features
        self.labels = labels
        self.h = h
        
    def gradient(self, X_val, feature):
        """
            Numerical Gradient Calculation by way of a CFD method.
            Inputs
            --------
            X_val : pandas.DataFrame with columns: features and values at
                which we want to take the numerical gradient.
            feature : feature by which we want to differentiate.
        """
        X_prime_plus = X_val.copy()
        X_prime_plus.loc[:,(feature)] = X_prime_plus[feature] + self.h
        X_prime_minus = X_val.copy()
        X_prime_minus.loc[:,(feature)] = X_prime_minus[feature] - self.h
        
        grad = (self.f(X_prime_plus) - self.f(X_prime_minus))/(2*self.h)
        
        return grad
    def feature_att(self, X):
        """
            Calculates the Gradients for all Entries in X, for each
            feature and label combination.
            
            Inputs
            --------
            X : pandas.DataFrame with columns:features and values at
                which we want to differentiate.
            Returns
            --------
            self.__atts : [np.array[...],np.array[...]] of gradients at
                each of the input points. Calculated for each label and stacked.
        """
        first_run = True
        for i,__label in enumerate(self.labels):
            grads = self.gradient(X, self.features[0])[:,i]
            for __feat in self.features[1:]:
                grads = np.vstack((grads,self.gradient(X, __feat)[:,i]))
            normalised_grads = np.abs(grads)/np.sum(np.abs(grads),axis=0)
            if first_run:
                self.__atts = grads.transpose()
                self.__normalised = normalised_grads.transpose()
                first_run = False
            else:
                self.__atts = [self.__atts, grads.transpose()]
                self.__normalised = [self.__normalised, normalised_grads.transpose()]
                        
        return self.__atts#, self.__normalised
    
class Bootstrapper():
    def __init__(self, model, data, features, labels, suffix, explainer_type, num_straps = 50, back_size = 100):
        self.explainer_type = explainer_type
        self.model = model
        self.data = data
        self.features = features
        self.labels = labels
        self.num_straps = num_straps
        self.back_size = back_size
        self.suffix = suffix
        
    def bootstrap(self, X):
        self.values = np.empty((self.num_straps, len(self.labels), len(self.features)))
        self.mean_std_arr = np.empty((2, len(self.labels), len(self.features)))
        for i in range(self.num_straps):
            background_i = shap.sample(self.data, self.back_size, random_state = np.random.randint(100))
            if self.explainer_type == 'kernel':
                exp_i = shap.KernelExplainer(self.model, background_i)
                shapper = exp_i.shap_values(X)
            elif self.explainer_type == 'sample':
                exp_i = shap.SampleExplainer(self.model, background_i)
                shapper = exp_i.shap_values(X)
            elif self.explainer_type == 'lime':
                exp_i = MyLime(self.model, background_i, mode="regression")
                shapper = exp_i.attributions(X)
            self.values[i,0,:] = shapper[0]
            self.values[i,1,:] = shapper[1]
        for i in range(len(self.labels)):
            for j in range(len(self.features)):
                self.mean_std_arr[0, i, j] = np.mean(self.values[:,i,j])
                self.mean_std_arr[1, i, j] = np.std(self.values[:,i,j])
            
        return self.mean_std_arr
    
    def to_df(self):
        self.bootstrap_df = self.x_list.copy()
        for k, col in enumerate(["mean", "std"]):
            for j in range(len(self.labels)):
                for i in range(len(self.features)):
                    self.bootstrap_df.insert(4 + i + j*len(self.features) + k*len(self.features)*len(self.labels), 
                                             self.features[i] + "_" + self.labels[j] + "_" + col, 
                                             self.bootstrap_array[:,k,j,i])
        if self.save:
            self.bootstrap_df.to_csv("Results/"+self.explainer_type+"/"+self.explainer_type+"bootstrap_vals_"+self.suffix+".csv")
        return self.bootstrap_df
    def calculate(self, num_samples = 10, save = True):
        self.x_list = self.data.iloc[np.sort(np.random.choice(self.data.shape[0], num_samples, replace =False))]
        self.bootstrap_array = np.empty((num_samples, 2, len(self.labels), len(self.features)))
        for i in tqdm(range(self.x_list.shape[0]), desc="Bootstrapping…", ascii=False, ncols=75):
            x_val = self.x_list.iloc[i,:]
            self.bootstrap_array[i,:,:,:] = self.bootstrap(x_val)
        return self.to_df()
    
class MyLime(shap.other.LimeTabular):
    def __init__(self, model, data, mode="classification"):
        self.model = model
        assert mode in ["classification", "regression"]
        self.mode = mode

        if str(type(data)).endswith("pandas.core.frame.DataFrame'>"):
            data = data.values
        self.data = data
        self.explainer = lime.lime_tabular.LimeTabularExplainer(data, mode=mode)
        self.out_dim = 1#self.model(data[0:1]).shape[1]
            
    def attributions(self, X, num_samples=50, num_features=None):
        try:
            num_features = X.shape[1] if num_features is None else num_features
        except:
            print('exception')
            num_features = 1
        if str(type(X)).endswith("pandas.core.frame.DataFrame'>"):
            X = X.values
            
        out = [np.zeros(X.shape) for j in range(len(self.model))]
        for i in tqdm(range(X.shape[0]), desc="Calculating Lime…", ascii=False, ncols=75):
            exp1 = self.explainer.explain_instance(X[i], self.model[0], labels=range(self.out_dim), 
                                                    num_features=num_features, num_samples=num_samples)
            exp2 = self.explainer.explain_instance(X[i], self.model[1], labels=range(self.out_dim), 
                                                    num_features=num_features, num_samples=num_samples)
            for k, v in exp1.local_exp[1]: 
                out[0][i,k] = v
            for k, v in exp2.local_exp[1]: 
                out[1][i,k] = v
          
        return out
    
"""
Define and Create Model
"""
def MLModel():
    opt = Adam(learning_rate=0.001, beta_1=0.7)
    loss='mse'
    model = Sequential([
        layers.Flatten(),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='sigmoid'),
        layers.Dense(32, activation='tanh'),
        layers.Dense(2)            
    ])
    model.compile(optimizer=opt, loss=loss)
    return model
  
    
def SimpleModel():
    opt = Adam(learning_rate=0.001, beta_1=0.7)
    loss='mse'
    model = Sequential([
        layers.Flatten(),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='tanh'),
        layers.Dense(2)            
    ])
    model.compile(optimizer=opt, loss=loss)
    return model
"""
Define Parameter Configuration to Model

    Parameters
    ----------
    alpha : float, linear stiffness
    beta  : float, non linearity in the restoring force
    gamma : float, amplitude of the periodic driving force
    delta : float, amount of damping
    omega : float, angular frequency of the periodic driving force
"""   
alpha = [-1.0, -0.1, 1.0]
beta = [-1.0, -0.1, 1.0]
gamma = [0.2, 0.28, 0.37, 0.5, 0.65]
delta = [0.3, 1.0]
omega = [0, 0.3, 1.2]

parameters = {'alpha': alpha, 'beta': beta, 'gamma': gamma, 'delta': delta, 'omega':omega}

parameter_grid = ParameterGrid(parameters)

for dict_param in parameter_grid:
    duffing = Duffing(parameters = dict_param)
    eom = duffing.eom
    suffix = duffing.suffix

    end_time = 100
    duffing.generate(10, samples = 100, end_time = end_time)
    duffing.scale_features()
    X = duffing.X_df[duffing.features]
    y = duffing.X_df[duffing.labels]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    # Create a basic model instance
    model = MLModel()

    """
    Train Model
    """

    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50),tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)]


    # pipe = make_pipeline(scaler, model)

    history=model.fit(X_train, y_train, steps_per_epoch=None, epochs=500, validation_split=0.2, batch_size=1024, shuffle=True, callbacks=None, verbose=1)

    loss = model.evaluate(X_test, y_test, verbose=1)
    print("Trained model, loss: {:5.2f}%".format(loss))

    model.save("Models/ml_model_"+suffix)
    
    def ml_x(X):
        return model.predict(X)[:,0]
    def ml_v(X):
        return model.predict(X)[:,1]
    
    # Make Simple ML Model
    simple_model = SimpleModel()

    """
    Train Model
    """

    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50),tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)]


    # pipe = make_pipeline(scaler, model)

    history=simple_model.fit(X_train, y_train, steps_per_epoch=None, epochs=500, validation_split=0.2, batch_size=1024, shuffle=True, callbacks=None, verbose=1)

    loss = simple_model.evaluate(X_test, y_test, verbose=1)
    print("Trained model, loss: {:5.2f}%".format(loss))

    simple_model.save("Models/simple_ml_model_"+suffix)
    
    def simple_ml_x(X):
        return simple_model.predict(X)[:,0]
    def simple_ml_v(X):
        return simple_model.predict(X)[:,1]

    explainers = ['lime']#["kernel", "sampling", "lime", "numeric"]
    true_lime = [duffing.predict_x, duffing.predict_v]
    ml_lime = [ml_x, ml_v]
    simple_lime = [simple_ml_x, simple_ml_v]
    models = {"simple" : simple_model.predict}#{"true" : duffing.predict, "ml" : model.predict, "simple" : simple_model.predict}
    lime_models = {"true" : true_lime, "ml" : ml_lime, "simple" : simple_lime}


    background = shap.sample(X_test, 100)
    choice = X.iloc[np.sort(np.random.choice(X_test.shape[0], 100, replace =False))]


    big_df = pd.DataFrame()
    for model_ in models:
        for explainer in explainers:
            print(explainer + model_)
            if explainer == "kernel":
                temp_explainer = shap.KernelExplainer(models[model_], background)
                temp_vals = temp_explainer.shap_values(choice)
            elif explainer == "sampling":
                temp_explainer = shap.SamplingExplainer(models[model_], background)
                temp_vals = temp_explainer.shap_values(choice)
            elif explainer == "lime":
                temp_explainer = MyLime(lime_models[model_], X, mode='regression')
                temp_vals = temp_explainer.attributions(choice)
            elif explainer == "numeric":
                temp_explainer = NumericExplainer(models[model_], duffing.features, duffing.labels, h = 0.001)
                temp_vals = temp_explainer.feature_att(choice)
            else:
                print("not a valid explainer type")
            big_df = big_df.append(duffing.vals_to_df(temp_vals, 
                                                            choice, save=False, explainer = explainer, suffix = suffix))

        
    big_df.to_csv("Results/explainer_dataframe_"+suffix+".csv")  



Generating Data…: 100%|███████████████████| 10/10 [00:00<00:00, 526.32it/s]


Epoch 1/500
1/1 [==============================] - 1s 550ms/step - loss: 86.3464 - val_loss: 83.2967
Epoch 2/500
1/1 [==============================] - 0s 21ms/step - loss: 84.8166 - val_loss: 81.7292
Epoch 3/500
1/1 [==============================] - 0s 21ms/step - loss: 83.2131 - val_loss: 80.0090
Epoch 4/500
1/1 [==============================] - 0s 21ms/step - loss: 81.4532 - val_loss: 78.0083
Epoch 5/500
1/1 [==============================] - 0s 23ms/step - loss: 79.4065 - val_loss: 75.5309
Epoch 6/500
1/1 [==============================] - 0s 21ms/step - loss: 76.8740 - val_loss: 72.4205
Epoch 7/500
1/1 [==============================] - 0s 22ms/step - loss: 73.6917 - val_loss: 68.8093
Epoch 8/500
1/1 [==============================] - 0s 23ms/step - loss: 69.9941 - val_loss: 65.0038
Epoch 9/500
1/1 [==============================] - 0s 21ms/step - loss: 66.0930 - val_loss: 61.3306
Epoch 10/500
1/1 [==============================] - 0s 23ms/step - loss: 62.3209 - val_loss: 58.328

1/1 [==============================] - 0s 22ms/step - loss: 16.6517 - val_loss: 17.0359
Epoch 83/500
1/1 [==============================] - 0s 22ms/step - loss: 16.3686 - val_loss: 16.7719
Epoch 84/500
1/1 [==============================] - 0s 23ms/step - loss: 16.0893 - val_loss: 16.5116
Epoch 85/500
1/1 [==============================] - 0s 22ms/step - loss: 15.8139 - val_loss: 16.2552
Epoch 86/500
1/1 [==============================] - 0s 22ms/step - loss: 15.5424 - val_loss: 16.0025
Epoch 87/500
1/1 [==============================] - 0s 23ms/step - loss: 15.2747 - val_loss: 15.7536
Epoch 88/500
1/1 [==============================] - 0s 22ms/step - loss: 15.0108 - val_loss: 15.5085
Epoch 89/500
1/1 [==============================] - 0s 22ms/step - loss: 14.7508 - val_loss: 15.2671
Epoch 90/500
1/1 [==============================] - 0s 22ms/step - loss: 14.4946 - val_loss: 15.0295
Epoch 91/500
1/1 [==============================] - 0s 25ms/step - loss: 14.2422 - val_loss: 14.7956
Epo

1/1 [==============================] - 0s 21ms/step - loss: 4.5577 - val_loss: 6.0570
Epoch 164/500
1/1 [==============================] - 0s 21ms/step - loss: 4.4965 - val_loss: 6.0042
Epoch 165/500
1/1 [==============================] - 0s 21ms/step - loss: 4.4365 - val_loss: 5.9524
Epoch 166/500
1/1 [==============================] - 0s 21ms/step - loss: 4.3776 - val_loss: 5.9016
Epoch 167/500
1/1 [==============================] - 0s 21ms/step - loss: 4.3197 - val_loss: 5.8518
Epoch 168/500
1/1 [==============================] - 0s 21ms/step - loss: 4.2629 - val_loss: 5.8030
Epoch 169/500
1/1 [==============================] - 0s 22ms/step - loss: 4.2071 - val_loss: 5.7551
Epoch 170/500
1/1 [==============================] - 0s 21ms/step - loss: 4.1523 - val_loss: 5.7082
Epoch 171/500
1/1 [==============================] - 0s 22ms/step - loss: 4.0986 - val_loss: 5.6622
Epoch 172/500
1/1 [==============================] - 0s 21ms/step - loss: 4.0458 - val_loss: 5.6171
Epoch 173/500


1/1 [==============================] - 0s 21ms/step - loss: 1.9828 - val_loss: 3.9513
Epoch 246/500
1/1 [==============================] - 0s 21ms/step - loss: 1.9704 - val_loss: 3.9426
Epoch 247/500
1/1 [==============================] - 0s 22ms/step - loss: 1.9583 - val_loss: 3.9341
Epoch 248/500
1/1 [==============================] - 0s 21ms/step - loss: 1.9464 - val_loss: 3.9257
Epoch 249/500
1/1 [==============================] - 0s 21ms/step - loss: 1.9348 - val_loss: 3.9176
Epoch 250/500
1/1 [==============================] - 0s 21ms/step - loss: 1.9234 - val_loss: 3.9097
Epoch 251/500
1/1 [==============================] - 0s 20ms/step - loss: 1.9123 - val_loss: 3.9020
Epoch 252/500
1/1 [==============================] - 0s 21ms/step - loss: 1.9014 - val_loss: 3.8945
Epoch 253/500
1/1 [==============================] - 0s 21ms/step - loss: 1.8907 - val_loss: 3.8872
Epoch 254/500
1/1 [==============================] - 0s 22ms/step - loss: 1.8803 - val_loss: 3.8801
Epoch 255/500


...

Epoch 410/500
1/1 [==============================] - 0s 23ms/step - loss: 1.0576 - val_loss: 2.6139
Epoch 411/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0575 - val_loss: 2.6141
Epoch 412/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0573 - val_loss: 2.6143
Epoch 413/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0572 - val_loss: 2.6145
Epoch 414/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0571 - val_loss: 2.6147
Epoch 415/500
1/1 [==============================] - 0s 23ms/step - loss: 1.0570 - val_loss: 2.6149
Epoch 416/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0569 - val_loss: 2.6151
Epoch 417/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0569 - val_loss: 2.6153
Epoch 418/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0568 - val_loss: 2.6155
Epoch 419/500
1/1 [==============================] - 0s 23ms/step - loss: 1.0567 - val_loss: 2.6157


Epoch 492/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0543 - val_loss: 2.6253
Epoch 493/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0543 - val_loss: 2.6254
Epoch 494/500
1/1 [==============================] - 0s 21ms/step - loss: 1.0543 - val_loss: 2.6254
Epoch 495/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0543 - val_loss: 2.6255
Epoch 496/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0543 - val_loss: 2.6256
Epoch 497/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0543 - val_loss: 2.6256
Epoch 498/500
1/1 [==============================] - 0s 22ms/step - loss: 1.0542 - val_loss: 2.6257
Epoch 499/500
1/1 [==============================] - 0s 21ms/step - loss: 1.0542 - val_loss: 2.6258
Epoch 500/500
4/4 [==============================] - 0s 1ms/step - loss: 1.9044
Trained model, loss:  1.90%
INFO:tensorflow:Assets written to: Models/ml_model_params_-1.0_-1.0_0.5_1.0_0.3\assets
Epoch

1/1 [==============================] - 0s 17ms/step - loss: 11.1962 - val_loss: 11.4273
Epoch 72/500
1/1 [==============================] - 0s 20ms/step - loss: 11.0068 - val_loss: 11.2524
Epoch 73/500
1/1 [==============================] - 0s 16ms/step - loss: 10.8207 - val_loss: 11.0805
Epoch 74/500
1/1 [==============================] - 0s 17ms/step - loss: 10.6380 - val_loss: 10.9118
Epoch 75/500
1/1 [==============================] - 0s 17ms/step - loss: 10.4584 - val_loss: 10.7462
Epoch 76/500
1/1 [==============================] - 0s 17ms/step - loss: 10.2821 - val_loss: 10.5835
Epoch 77/500
1/1 [==============================] - 0s 18ms/step - loss: 10.1088 - val_loss: 10.4238
Epoch 78/500
1/1 [==============================] - 0s 17ms/step - loss: 9.9385 - val_loss: 10.2669
Epoch 79/500
1/1 [==============================] - 0s 16ms/step - loss: 9.7712 - val_loss: 10.1129
Epoch 80/500
1/1 [==============================] - 0s 16ms/step - loss: 9.6068 - val_loss: 9.9616
Epoch 8

1/1 [==============================] - 0s 21ms/step - loss: 2.9254 - val_loss: 3.9649
Epoch 154/500
1/1 [==============================] - 0s 18ms/step - loss: 2.8829 - val_loss: 3.9289
Epoch 155/500
1/1 [==============================] - 0s 18ms/step - loss: 2.8411 - val_loss: 3.8936
Epoch 156/500
1/1 [==============================] - 0s 22ms/step - loss: 2.8003 - val_loss: 3.8591
Epoch 157/500
1/1 [==============================] - 0s 25ms/step - loss: 2.7602 - val_loss: 3.8254
Epoch 158/500
1/1 [==============================] - 0s 18ms/step - loss: 2.7210 - val_loss: 3.7924
Epoch 159/500
1/1 [==============================] - 0s 18ms/step - loss: 2.6825 - val_loss: 3.7602
Epoch 160/500
1/1 [==============================] - 0s 18ms/step - loss: 2.6448 - val_loss: 3.7287
Epoch 161/500
1/1 [==============================] - 0s 19ms/step - loss: 2.6079 - val_loss: 3.6978
Epoch 162/500
1/1 [==============================] - 0s 19ms/step - loss: 2.5718 - val_loss: 3.6677
Epoch 163/500


KeyboardInterrupt: 

In [79]:
choice

,x0,v0,t,rand,energy,alpha,beta,delta,gamma,omega
0,0.139358,0.712804,0.000000,0.790637,0.655385,0.0,0.0,0.0,0.0,0.0
1,0.139358,0.712804,0.010101,0.606258,0.655385,0.0,0.0,0.0,0.0,0.0
2,0.139358,0.712804,0.020202,0.926883,0.655385,0.0,0.0,0.0,0.0,0.0
3,0.139358,0.712804,0.030303,0.651415,0.655385,0.0,0.0,0.0,0.0,0.0
4,0.139358,0.712804,0.040404,0.915532,0.655385,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
95,0.139358,0.712804,0.959596,0.220198,0.655385,0.0,0.0,0.0,0.0,0.0
96,0.139358,0.712804,0.969697,0.711541,0.655385,0.0,0.0,0.0,0.0,0.0
97,0.139358,0.712804,0.979798,0.809980,0.655385,0.0,0.0,0.0,0.0,0.0
98,0.139358,0.712804,0.989899,0.348737,0.655385,0.0,0.0,0.0,0.0,0.0


In [80]:
simple_ml_x(choice)

array([9.849846, 9.850744], dtype=float32)

In [83]:
simple_model.predict(X)[:,0]

array([9.849846 , 9.8491745, 9.850267 , 9.849457 , 9.850284 , 9.850151 ,
       9.848635 , 9.845756 , 9.848282 , 9.8497505, 9.849776 , 9.849548 ,
       9.847925 , 9.849514 , 9.848741 , 9.850578 , 9.850393 , 9.850145 ,
       9.848126 , 9.848328 , 9.846788 , 9.850218 , 9.84757  , 9.849465 ,
       9.848349 , 9.85062  , 9.849716 , 9.850016 , 9.850345 , 9.848337 ,
       9.850201 , 9.850073 , 9.848885 , 9.850775 , 9.8502865, 9.850485 ,
       9.850744 , 9.850402 , 9.8495865, 9.848843 , 9.850124 , 9.84944  ,
       9.850088 , 9.850821 , 9.849911 , 9.849275 , 9.85006  , 9.850876 ,
       9.849903 , 9.850647 , 9.849092 , 9.850195 , 9.850116 , 9.849797 ,
       9.850578 , 9.850525 , 9.850784 , 9.850189 , 9.850166 , 9.850032 ,
       9.850174 , 9.850711 , 9.850538 , 9.849928 , 9.850351 , 9.850363 ,
       9.8508835, 9.850903 , 9.850264 , 9.851049 , 9.850506 , 9.851051 ,
       9.850713 , 9.850189 , 9.850674 , 9.850919 , 9.850952 , 9.850872 ,
       9.850822 , 9.850904 , 9.850935 , 9.8504715, 

In [69]:
simple_ml_v(X)

array([9.8491745, 9.850441 ], dtype=float32)

In [57]:
simple_model.predict(choice)

array([[9.850363, 9.850977],
       [9.851051, 9.851259]], dtype=float32)

In [54]:
big_df

,xt_x0,xt_v0,xt_t,xt_rand,xt_energy,xt_alpha,xt_beta,xt_delta,xt_gamma,xt_omega,...,v0,t,rand,energy,alpha,beta,delta,gamma,omega,explainer
0,0.0,0.0,0.000036,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,...,0.712804,0.656566,0.246857,0.655385,0.0,0.0,0.0,0.0,0.0,lime
1,0.0,0.0,0.000037,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,...,0.712804,0.717172,0.977241,0.655385,0.0,0.0,0.0,0.0,0.0,lime


In [46]:
duffing.features

['x0', 'v0', 't', 'rand', 'energy', 'alpha', 'beta', 'delta', 'gamma', 'omega']